In [1]:
%pwd

'd:\\Ml Dl\\Project\\Text-Summaraizer\\reasearch'

In [2]:
cd ../

d:\Ml Dl\Project\Text-Summaraizer


c:\Users\ronak\anaconda3\envs\text\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd


'd:\\Ml Dl\\Project\\Text-Summaraizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path : Path
    tokenizer_name : Path
    


In [ ]:
from src.utils.common import read_yaml
from src.utils.common import create_directories
from src.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(self,config_filepath= CONFIG_FILE_PATH,params_filepath= PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation(self) -> DataTransformationConfig:
        config = self.config.data_transformation                 # Extracts only the data_ingestion part of config.yaml.

        create_directories([config.root_dir])               # Create data_ingestion.root directory

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )

        return data_transformation_config

In [8]:
import os
from src.logger import logger
# from src.entity import DataTransformationConfig
from datasets import load_from_disk
class Data_Transformation:
    def __init__(self,config: DataTransformationConfig):
        self.config= config
        self.tokenizer = config.tokenizer_name

    def tokenize_function(self,example):
        # Tokenize dialogue (input)
        inputs = self.tokenizer(example["dialogue"], max_length=512, truncation=True)
        # Tokenize summary (target)
        targets = self.tokenizer(example["summary"], max_length=512, truncation=True)

        inputs["labels"] = targets["input_ids"]
        return {
            "input_ids":inputs["input_ids"],
            "attention_mask":inputs["attention_mask"],
            "labels":targets["input_ids"]
        }
    
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.tokenize_function,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

